# Full Solution-Milestone-1

In this part, instead of visiting a website and downloading the data to our local computer, we directly access to `yahoo finance` and retrieve the data as  we have learned in the previous lesson. To compare the performance of the companies, we will run CAPM analyses with four companies operating in tech sector. These companies are `Apple`, `IBM`,`Microsoft`, `Intel` and the market these stocks are listed is `S&P-500`. We use tickers to retreive data from yahoo finance:
 
 * AAPL: Apple
 * IBM: IBM
 * MSFT: Microsoft
 * INTC: Intel
 * ^GSPC: S&P-500

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
import warnings
import yfinance as yf
import seaborn as sns; sns.set()
yf.pdr_override() 
%matplotlib inline
warnings.filterwarnings('ignore')

The analysis covers the business day for the period of 01-01-2021 and 01-04-2021 with monthly stock price data.

In [2]:
stocks = ['AAPL','IBM','MSFT','INTC','^GSPC']
start = datetime.datetime(2021,1,1)
end = datetime.datetime(2021,4,1)
stock_prices = yf.download(stocks,start=start,end = end, interval='1d')

[*********************100%***********************]  5 of 5 completed


In [3]:
stock_prices.head()

Adj Close                                                  \
                  AAPL         IBM       INTC        MSFT        ^GSPC   
Date                                                                     
2020-12-31  132.267349  122.847458  49.216290  221.397675  3756.070068   
2021-01-04  128.997803  120.954201  49.068108  216.689423  3700.649902   
2021-01-05  130.592697  123.101204  49.996723  216.898438  3726.860107   
2021-01-06  126.196747  126.175316  50.480778  211.274414  3748.139893   
2021-01-07  130.502991  125.882545  51.557571  217.286652  3803.790039   

                 Close                                                  ...  \
                  AAPL         IBM       INTC        MSFT        ^GSPC  ...   
Date                                                                    ...   
2020-12-31  132.690002  125.879997  49.820000  222.419998  3756.070068  ...   
2021-01-04  129.410004  123.940002  49.669998  217.690002  3700.649902  ...   
2021-01-05  131.009995  126.139999  50.610001  217.899994  3726.860107  ...   
2021-01-06  126.599998  129.289993  51.099998  212.250000  3748.139893  ...   
2021-01-07  130.919998  128.990005  52.189999  218.289993  3803.790039  ...   

                  Open                                                  \
                  AAPL         IBM       INTC        MSFT        ^GSPC   
Date                                                                     
2020-12-31  134.080002  124.220001  48.750000  221.699997  3733.270020   
2021-01-04  133.520004  125.849998  49.889999  222.529999  3764.610107   
2021-01-05  128.889999  125.010002  49.450001  217.259995  3698.020020   
2021-01-06  127.720001  126.900002  50.439999  212.169998  3712.199951   
2021-01-07  128.360001  130.039993  51.490002  214.039993  3764.709961   

               Volume                                           
                 AAPL      IBM      INTC      MSFT       ^GSPC  
Date                                                            
2020-12-31   99116600  3574400  28576800  20942100  3172510000  
2021-01-04  143301900  5179200  46102500  37130100  5006680000  
2021-01-05   97664900  6114600  24866600  23823000  4582620000  
2021-01-06  155088000  7956700  36809600  35930700  6049970000  
2021-01-07  109578200  4507400  32092300  27694500  5080870000  

[5 rows x 30 columns]

In [4]:
stock_prices=stock_prices['Close']

In [5]:
stock_prices.dropna(inplace=True)

Summary statistics is a very first step to explore the dataset. Main statistics such as mean, standard deviation, minimum, maximum etc., can be calculated via `.describe()` in Python. Initial observation tells us that the price level and volatility (represented by standard deviation) of S&P-500 is quite different from the rest of the stocks.

In [6]:
stock_prices.describe()

,AAPL,IBM,INTC,MSFT,^GSPC
count,62.000000,62.000000,62.000000,62.000000,62.000000
mean,128.427903,125.195968,59.438226,231.866613,3863.938563
std,6.726222,4.862788,4.248762,9.164177,71.994513
min,116.360001,118.580002,49.669998,212.250000,3700.649902
25%,121.984999,120.754999,57.107501,226.894997,3810.167542
50%,128.315002,124.060001,60.530001,233.525002,3873.589966
75%,133.752502,128.985004,62.827500,237.677505,3915.557556
max,143.160004,136.380005,65.779999,244.990005,3974.540039


As the caret sign `^` in the ticker of S&P-500 might cause problem in coding, I prefer rename this column before proceeding:

In [7]:
stock_prices=stock_prices.rename({'^GSPC':'GSPC'}, axis='columns')

Now, after having the return for three stock and one market index, extracting risk free rate allows us to compute excess return. To access the risk free rate data, we use `Fred API` as we have learned previosly. Please note that there is no single risk free rate, different researchers apply different risk free rate but the common property of these rates is that they are issued by a government, which makes them safe heaven to invest. Here, we select `3-Month Treasury Constant Maturity Rate`, which is short-term T-Bill with relatively low and safe return.

Practically speaking, risk-free rate is hypothetical term, because even the safest investments carry a very small amount of risk. However, it is not wrong to say that government-issued debt is much safer compared to comporate-issued one. That is being said, the interest rate on a three-month U.S. Treasury bill is useful proxy for the risk-free rate for U.S.-based investors. 

In [8]:
from fredapi import Fred
fred = Fred(api_key='type your FRED API key')

In [9]:
fred.search('risk free')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
DGS10,DGS10,2021-08-05,2021-08-05,10-Year Treasury Constant Maturity Rate,1962-01-02,2021-08-03,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-08-04 15:23:08-05:00,97,For further information regarding treasury con...
GS10,GS10,2021-08-05,2021-08-05,10-Year Treasury Constant Maturity Rate,1953-04-01,2021-07-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2021-08-02 15:23:21-05:00,75,Averages of business days. For further informa...
WGS10YR,WGS10YR,2021-08-05,2021-08-05,10-Year Treasury Constant Maturity Rate,1962-01-05,2021-07-30,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2021-08-02 15:24:09-05:00,61,Averages of business days. For further informa...
DGS1,DGS1,2021-08-05,2021-08-05,1-Year Treasury Constant Maturity Rate,1962-01-02,2021-08-03,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-08-04 15:23:06-05:00,80,For further information regarding treasury con...
WGS1YR,WGS1YR,2021-08-05,2021-08-05,1-Year Treasury Constant Maturity Rate,1962-01-05,2021-07-30,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2021-08-02 15:24:13-05:00,56,Averages of business days. For further informa...
GS1,GS1,2021-08-05,2021-08-05,1-Year Treasury Constant Maturity Rate,1953-04-01,2021-07-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2021-08-02 15:23:37-05:00,51,Averages of business days. For further informa...
DGS5,DGS5,2021-08-05,2021-08-05,5-Year Treasury Constant Maturity Rate,1962-01-02,2021-08-03,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-08-04 15:23:21-05:00,78,For further information regarding treasury con...
DGS30,DGS30,2021-08-05,2021-08-05,30-Year Treasury Constant Maturity Rate,1977-02-15,2021-08-03,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-08-04 15:22:11-05:00,79,Yields on actively traded non-inflation-indexe...
GS5,GS5,2021-08-05,2021-08-05,5-Year Treasury Constant Maturity Rate,1953-04-01,2021-07-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2021-08-02 15:23:34-05:00,52,Averages of business days. For further informa...


In [10]:
risk_free=fred.get_series('DGS3MO')
risk_free=risk_free['2021-01-01':'2021-04-01']